<a href="https://colab.research.google.com/github/akheel8743/Brolly1/blob/main/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1.Installing Required Libraries

In [1]:
!pip install requests beautifulsoup4 pandas

# Step 2.Importing Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import os

#Step 3 - Save data to CSV and JSON

In [5]:
df = pd.DataFrame(books)
df.to_csv("books.csv", index=False)
with open("books.json", "w") as f:
    # Indent the json.dump line to be inside the 'with' block
    json.dump(books, f, indent=2)

print("Data scraped and saved!")

Data scraped and saved!


# Step 4-Generate index.html

In [6]:
base_url = "https://books.toscrape.com/catalogue/"
start_url = "https://books.toscrape.com/catalogue/page-1.html"
books = []

while start_url:
    response = requests.get(start_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # The scraping logic for a single page goes here
    for book in soup.select("article.product_pod"):
        title = book.h3.a["title"]
        price = book.select_one(".price_color").text
        rel_link = book.h3.a["href"]
        image_url = "https://books.toscrape.com/" + book.img["src"].replace("../", "")

        books.append({
            "title": title,
            "price": price,
            "image_url": image_url,
            "detail_page": base_url + rel_link
        })

    next_btn = soup.select_one("li.next > a")
    if next_btn:
        next_page = next_btn["href"]
        start_url = "https://books.toscrape.com/catalogue/" + next_page
    else:
        start_url = None # Set start_url to None to exit the loop

#Step 5 - Create HTML

In [7]:
#Create HTML content and save it to a file:

html_template = """

<!DOCTYPE html>
<html>
<head>
<title>Books Store</title>
<link rel="stylesheet" href="styles.css">
</head>
<body>
<h1>Books Store</h1>
<div class="grid">
{}
</div>
</body>
</html>
"""
book_cards = ""
for i in range(10): # You can increase this to all books if needed
    # Indent the following lines to be inside the for loop
    title = df.iloc[i]['title']
    price = df.iloc[i]['price']
    image_url = df.iloc[i]['image_url']
    book_cards += f"""
    <div>
    <img src="{image_url}" width="100">
    <p>{title}</p>
    <p>{price}</p>
    </div>
    """

full_html = html_template.format(book_cards)

with open("index.html", "w", encoding="utf-8") as f:
    f.write(full_html)

print("index.html generated!")

index.html generated!


# Step 6 - Create  Style CSS File

In [8]:
#Create a CSS file for styling:

css_code = """
body {
font-family: Arial;
background: #f4f4f4;
padding: 20px;
}
.grid {
display: grid;
grid-template-columns: repeat(auto-fill, minmax(150px, 1fr));
gap: 20px;
}
.grid div {
background: white;
padding: 10px;
border: 1px solid #ccc;
}
img {
max-width: 100%;
}
"""

with open("styles.css", "w") as f:
     f.write(css_code)

print("styles.css generated!")

styles.css generated!
